In [1]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 566 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import yfinance as yf
import pandas as pd

## **Page 1 Title: Stock Ticker Price History**

In [4]:
# choose any 1 stock ticker and get price history for it over a specified period of time
# also tells you the return of the stock over the specified period of time

import matplotlib.pyplot as plt
import yfinance as yf
import plotly.express as px 
from datetime import datetime

ticker = "TSLA"
start_date = '2018-12-11'
end_date = '2022-12-11'

def get_ticker_info(ticker):
  stock_info_dict = dict()
  stock_ticker = yf.Ticker(ticker)
  year_low = stock_ticker.info['fiftyTwoWeekLow']
  year_high = stock_ticker.info['fiftyTwoWeekHigh']
  #price = stock_ticker.info['open']
  quote_type = stock_ticker.info['quoteType']
  long_name = stock_ticker.info['longName']
  business_summary = stock_ticker.info['longBusinessSummary']
  volume = stock_ticker.info['volume']

  stock_info_dict = {"Full Name": long_name, "Security Type": quote_type, "52 Week High": year_high, "52 Week Low": year_low, "Volume": volume}

  if quote_type == "EQUITY":
    div_yield = stock_ticker.info['dividendYield']
    if div_yield is None:
      div_yield = 0
    stock_info_dict["Dividend Yield"] = div_yield
    price = stock_ticker.info['currentPrice']
    stock_info_dict["Current Price"] = price
  elif quote_type == "ETF":
    div_yield = stock_ticker.info['yield']
    if div_yield is None:
      div_yield = 0
    stock_info_dict["Dividend Yield"] = div_yield
    price = stock_ticker.info['open']
    stock_info_dict["Current Price"] = price
    top_holdings = stock_ticker.info['holdings']
    stock_info_dict["Top 10 Holdings"] = top_holdings
    sector_weights = stock_ticker.info['sectorWeightings']
    stock_info_dict["Sector Weighting"] = sector_weights

  #stock_info_df = pd.DataFrame(stock_info_dict, index = range(1)).reset_index(drop=True)
  #curr_price_column = stock_info_df.pop('Current Price')
  #stock_info_df.insert(2, 'Current Price', curr_price_column)
  return stock_info_dict
  
def get_ticker_price_chart(ticker, start_date, end_date):
  start_dt = datetime.strptime(start_date, '%Y-%m-%d')
  end_dt = datetime.strptime(end_date, '%Y-%m-%d')
  day_count = (end_dt - start_dt).days

  price_data = yf.download(ticker, start_date, end_date).reset_index()
  start_price = price_data.iloc[0]["Close"]
  #end_price = price_data.iloc[-1]["Close"]
  price_data["% Change"] = price_data["Close"].apply(lambda x: round(((x / start_price) - 1) * 100, 2))
  price_data["Return"] = price_data["Close"].apply(lambda x: round(x - start_price, 2))
  #total_return = round(((end_price - start_price)/start_price)*100, 2)
  print("Total return over the specified time period = ${} ({}%)".format(price_data.iloc[-1]["Return"], price_data.iloc[-1]["% Change"]))
  fig = px.line(price_data, x="Date", y="Close", labels={'Close': 'Price'}, custom_data = ["% Change", "Return"])
  fig.update_layout(yaxis_title = 'Price of {}'.format(ticker), hovermode="x unified")
  fig.update_traces(hovertemplate = "Price: $%{y} <br>Total Return: $%{customdata[1]} (%{customdata[0]}%)")
  return fig.show()

def display_holdings_bar_chart(holdings_dict):
  holdings_df = pd.DataFrame(holdings_dict).rename(columns = {'symbol': "Ticker", "holdingName": "Name", "holdingPercent": "Holding Percent"})
  holdings_df["Holding Percent"] = holdings_df["Holding Percent"] * 100
  fig = px.bar(holdings_df, x='Holding Percent', y='Name', title='{} Top 10 Holdings'.format(ticker))
  display(holdings_df)
  return fig.show()


get_ticker_price_chart(ticker, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
Total return over the specified time period = $154.6 (632.29%)


## **Page 2 Title: Annual Portfolio Dividend Payment**

In [11]:
# Annual Portfolio Dividend Payment
import plotly.express as px
import yfinance as yf
import pandas as pd

stock_list = ["JEPI", "JEPQ", "QQQ", "ICLN"]
shares_list = [50, 50, 50, 60]

def get_div_yield(ticker, num_of_shares):
  stock_ticker = yf.Ticker(ticker)
  div_rate = stock_ticker.info['dividendRate']
  price = stock_ticker.info['open']
  if stock_ticker.info['legalType'] == 'Exchange Traded Fund':
    div_rate = stock_ticker.info['yield'] * price
  if div_rate != None:
    annual_div = num_of_shares * div_rate
  else:
    annual_div = 0
  return [ticker, price, div_rate, num_of_shares, annual_div]

def dividend_df(stock_list, shares_list):
  div_yield = map(get_div_yield, stock_list, shares_list)
  div_list = list(div_yield)
  div_df = pd.DataFrame(div_list, columns = ["Ticker", "Price", "Annual Div Per Share ($)", "Total Shares", "Annual Total Dividend ($)"])
  div_df["Annual Div Per Share ($)"] = div_df["Annual Div Per Share ($)"].fillna(0)
  div_df["Annual Div Per Share ($)"] = round(div_df["Annual Div Per Share ($)"], 2)
  div_df["Annual Total Dividend ($)"] = round(div_df["Annual Total Dividend ($)"], 2)
  total_account_div = round(sum(div_df["Annual Total Dividend ($)"]), 2)
  print("Total amount of dividends collected per year: ${}".format(total_account_div))
  return div_df

def pie_chart(div_df):
  fig = px.pie(div_df, values='Annual Total Dividend ($)', names='Ticker', title='Dividends per Ticker')
  fig.update_layout(title_x = 0.5)
  return fig.show()
div_df = dividend_df(stock_list, shares_list)
display(div_df)
pie_chart(div_df)

Total amount of dividends collected per year: $757.93


,Ticker,Price,Annual Div Per Share ($),Total Shares,Annual Total Dividend ($)
0,JEPI,55.85,5.92,50,296.01
1,JEPQ,43.29,6.88,50,343.94
2,QQQ,282.81,2.01,50,100.40
3,ICLN,20.63,0.29,60,17.58


## **Page 3 Title: Stock Ticker Comparison**

In [159]:
# Compare a stock with drip or without drip. Or compare multiple stocks with or without DRIP
# Stock Ticker Comparison

import matplotlib.pyplot as plt
import yfinance as yf
import plotly.express as px 
from datetime import datetime

tickers = ["MCD", "VOO", "VGT"]
start_date = '2021-12-06'
end_date = '2022-12-08'
total_investment = 150
DRIP = True

def get_returns(tickers, start_date, end_date, total_investment, DRIP):

  start_dt = datetime.strptime(start_date, '%Y-%m-%d')
  end_dt = datetime.strptime(end_date, '%Y-%m-%d')
  day_count = (end_dt - start_dt).days

  data = yf.download(tickers, start_date, end_date)

  if len(tickers) == 1:
    stock_ticker = yf.Ticker(tickers[0])
    quote_type = stock_ticker.info['quoteType']
    if quote_type == "EQUITY":
      div_yield = stock_ticker.info['dividendYield']
    elif quote_type == "ETF":
      div_yield = stock_ticker.info['yield']

    if div_yield is not None:
      daily_returns = data["Close"].pct_change()
      monthly_returns = data["Close"].resample('M').ffill().pct_change()
      daily_returns_adj = data["Adj Close"].pct_change()
      monthly_returns_adj = data["Adj Close"].resample('M').ffill().pct_change()
      cum_returns = (daily_returns + 1).cumprod()
      cum_returns_adj = (daily_returns_adj + 1).cumprod()
      cum_returns_tot_invest = (cum_returns * total_investment).reset_index()
      cum_returns_tot_invest_adj = (cum_returns_adj * total_investment).reset_index()
      combined_tot_returns = pd.merge(cum_returns_tot_invest_adj, cum_returns_tot_invest, on = ["Date"])
      combined_tot_returns.loc[0, list(combined_tot_returns.columns)[1:]] = total_investment
      return combined_tot_returns

    else:
      daily_returns = data["Close"].pct_change()
      monthly_returns = data["Close"].resample('M').ffill().pct_change()
      cum_returns = (daily_returns + 1).cumprod()
      cum_returns_tot_invest = (cum_returns * total_investment).reset_index()
      cum_returns_tot_invest.loc[0, list(cum_returns_tot_invest.columns)[1:]] = total_investment
      return cum_returns_tot_invest

  # return without DRIP
  if DRIP == False:
    daily_returns = data["Close"].pct_change()
    monthly_returns = data["Close"].resample('M').ffill().pct_change()
    cum_returns = (daily_returns + 1).cumprod()
    cum_returns_tot_invest = (cum_returns * total_investment).reset_index()
    cum_returns_tot_invest.loc[0, list(cum_returns_tot_invest.columns)[1:]] = total_investment
    return cum_returns_tot_invest

  # return with DRIP
  elif DRIP == True:
    daily_returns_adj = data["Adj Close"].pct_change()
    monthly_returns_adj = data["Adj Close"].resample('M').ffill().pct_change()
    cum_returns_adj = (daily_returns_adj + 1).cumprod()
    cum_returns_tot_invest_adj = (cum_returns_adj * total_investment).reset_index()
    col_list = list(cum_returns_tot_invest_adj.columns)[1:]
    cum_returns_tot_invest_adj.loc[0, col_list] = total_investment
    return cum_returns_tot_invest_adj


def plot_returns(tickers, total_investment, DRIP, cum_total_returns):
  display(cum_total_returns)
  if (DRIP == True) & (len(tickers) > 1):
    fig = px.line(cum_total_returns, x="Date", y=cum_total_returns.columns[1:len(tickers)+1], labels={"variable": "Ticker", 'value': 'Investment Value'})
    fig.update_layout(yaxis_title = 'Growth of ${}'.format(total_investment), hovermode="x unified")
    fig.show()

  elif (DRIP == False) & (len(tickers) > 1):
    fig = px.line(cum_total_returns, x="Date", y=cum_total_returns.columns[1:], labels={"variable": "Ticker", 'value': 'Investment Value'})
    fig.update_layout(yaxis_title = 'Growth of ${}'.format(total_investment), hovermode="x unified")
    fig.show()

  elif len(tickers) == 1:
    ticker_name = tickers[0]
    if "Adj Close" not in cum_total_returns.columns:
      cum_total_returns = cum_total_returns.rename(columns={"Close": ticker_name})
      fig = px.line(cum_total_returns, x="Date", y=cum_total_returns.columns[1:], labels={"variable": "Ticker", 'value': 'Investment Value'})
      fig.update_layout(yaxis_title = 'Growth of ${}'.format(total_investment), hovermode="x unified")
      fig.show()
    else:
      cum_total_returns = cum_total_returns.rename(columns={"Close": "{} Without DRIP".format(ticker_name), "Adj Close": "{} With DRIP".format(ticker_name)})
      fig = px.line(cum_total_returns, x="Date", y=cum_total_returns.columns[1:], labels={"variable": "Key", 'value': 'Investment Value'})
      fig.update_layout(yaxis_title = 'Growth of ${}'.format(total_investment), hovermode="x unified")
      fig.show()

  elif len(tickers) == 0:
    print("Please input Stock Tickers")

cum_total_returns = get_returns(tickers, start_date, end_date, total_investment, DRIP)
plot_returns(tickers, total_investment, DRIP, cum_total_returns)

[*********************100%***********************]  3 of 3 completed


,Date,MCD,VGT,VOO
0,2021-12-06,150.000000,150.000000,150.000000
1,2021-12-07,152.416039,155.424021,153.083079
2,2021-12-08,152.222541,156.461077,153.513363
3,2021-12-09,153.841037,154.431342,152.467885
4,2021-12-10,155.383318,156.935259,153.922299
...,...,...,...,...
249,2022-12-01,163.917588,120.033762,135.267593
250,2022-12-02,163.917588,119.359968,135.072565
251,2022-12-05,162.832400,117.087661,132.667261
252,2022-12-06,162.940315,114.543774,130.749520


## **Page 4 Title: Growth of Shares with DRIP**

In [12]:
# hypothetical growth with DRIP for a fixed annual growth rate
# Growth of Shares with DRIP

initial_shares = 20
initial_share_price = 100
avg_annual_growth_rate = 0.08
div_yield = 0.03
years = 10

def share_growth(initial_shares, initial_share_price, avg_annual_growth_rate, div_yield, years):
  investment_dict = dict()
  share_price = initial_share_price
  shares = initial_shares
  total_div_earned = 0
  yearly_div_earned = 0
  portfolio_worth = 0

  for i in range(1, years+1):
    share_price = share_price * (1 + avg_annual_growth_rate)
    # per 1 share
    annual_div_payout = div_yield * share_price
    # shares with DRIP
    shares = shares + (shares*div_yield)
    # total div earned
    yearly_div_earned = shares * annual_div_payout
    total_div_earned += yearly_div_earned
    investment_dict["Year {}".format(i)] = {"Share Price": round(share_price, 2), "Shares": round(shares, 2), "Annual Dividends Earned": round(yearly_div_earned, 2), "Total Dividends Earned": round(total_div_earned, 2)}

  return pd.DataFrame(investment_dict).T

share_growth(initial_shares, initial_share_price, avg_annual_growth_rate, div_yield, years)

,Share Price,Shares,Annual Dividends Earned,Total Dividends Earned
Year 1,108.00,20.60,66.74,66.74
Year 2,116.64,21.22,74.25,140.99
Year 3,125.97,21.85,82.59,223.58
Year 4,136.05,22.51,91.87,315.46
Year 5,146.93,23.19,102.20,417.66
Year 6,158.69,23.88,113.69,531.35
Year 7,171.38,24.60,126.47,657.81
Year 8,185.09,25.34,140.68,798.50
Year 9,199.90,26.10,156.49,954.99
Year 10,215.89,26.88,174.08,1129.07


## **Page 5 Title: Periodic Contribution Investment Growth**

In [13]:
# tells how much an initial investment would be worth given a fixed annual growth rate and can include additional contributions
# Periodic Contribution Investment Growth

initial_investment = 1000
avg_annual_growth_rate = 0.1
additional_contributions = 150
contribution_rate = 'annual'
years = 15


def periodic_contribution_growth(initial_investment, avg_annual_growth_rate, additional_contributions, contribution_rate, years):

  total_return = 0
  total_contributions = 0
  portfolio_worth = initial_investment
  investment_dict = dict()

  if contribution_rate == "annual":
    for i in range(1, (years+1)):
      total_contributions += additional_contributions
      portfolio_worth = portfolio_worth * (1 + avg_annual_growth_rate)
      portfolio_worth += additional_contributions
      total_return = portfolio_worth - initial_investment - total_contributions 
      investment_dict[i] = {"Portfolio Worth": round(portfolio_worth, 2), "Total Return": round(total_return, 2), "Annual Contribution": round(additional_contributions, 2), "Total Contribution": round(total_contributions, 2)}
    growth_df = pd.DataFrame(investment_dict).T

  elif contribution_rate == "semi-annual":
    avg_annual_growth_rate = avg_annual_growth_rate / 2
    for i in range(1, (years+1) * 2):
      total_contributions += additional_contributions
      portfolio_worth = portfolio_worth * (1 + avg_annual_growth_rate)
      portfolio_worth += additional_contributions
      total_return = portfolio_worth - initial_investment - total_contributions 
      investment_dict[i] = {"Portfolio Worth": round(portfolio_worth, 2), "Total Return": round(total_return, 2), "Annual Contribution": round(additional_contributions*2, 2), "Total Contribution": round(total_contributions, 2)}
    growth_df = pd.DataFrame(investment_dict).T.iloc[1::2].reset_index(drop=True)
    growth_df.index = growth_df.index + 1

  elif contribution_rate == "monthly":
    avg_annual_growth_rate = avg_annual_growth_rate / 12
    for i in range(1, (years+1) * 12):
      total_contributions += additional_contributions
      portfolio_worth = portfolio_worth * (1 + avg_annual_growth_rate)
      portfolio_worth += additional_contributions
      total_return = portfolio_worth - initial_investment - total_contributions 
      investment_dict[i] = {"Portfolio Worth": round(portfolio_worth, 2), "Total Return": round(total_return, 2), "Annual Contribution": round(additional_contributions*12, 2), "Total Contribution": round(total_contributions, 2)}
    growth_df = pd.DataFrame(investment_dict).T.iloc[11::12].reset_index(drop=True)
    growth_df.index = growth_df.index + 1

  elif contribution_rate == "bi-weekly":
    avg_annual_growth_rate = avg_annual_growth_rate / 26
    for i in range(1, (years+1) * 26):
      total_contributions += additional_contributions
      portfolio_worth = portfolio_worth * (1 + avg_annual_growth_rate)
      portfolio_worth += additional_contributions
      total_return = portfolio_worth - initial_investment - total_contributions 
      investment_dict[i] = {"Portfolio Worth": round(portfolio_worth, 2), "Total Return": round(total_return, 2), "Annual Contribution": round(additional_contributions*26, 2), "Total Contribution": round(total_contributions, 2)}
    growth_df = pd.DataFrame(investment_dict).T.iloc[25::26].reset_index(drop=True)
    growth_df.index = growth_df.index + 1

  elif contribution_rate == "weekly":
    avg_annual_growth_rate = avg_annual_growth_rate / 52
    for i in range(1, (years+1) * 52):
      total_contributions += additional_contributions
      portfolio_worth = portfolio_worth * (1 + avg_annual_growth_rate)
      portfolio_worth += additional_contributions
      total_return = portfolio_worth - initial_investment - total_contributions
      investment_dict[i] = {"Portfolio Worth": round(portfolio_worth, 2), "Total Return": round(total_return, 2), "Annual Contribution": round(additional_contributions*52, 2), "Total Contribution": round(total_contributions, 2)}
    growth_df = pd.DataFrame(investment_dict).T.iloc[51::52].reset_index(drop=True)
    growth_df.index = growth_df.index + 1

  growth_df["Initial Investment"] = initial_investment
  return growth_df


def bar_contr_growth(growth_df):
  growth_df_subset = growth_df[["Initial Investment", "Total Contribution", "Total Return"]]
  fig = px.bar(growth_df_subset, 
              x = growth_df_subset.index,
              y = [c for c in growth_df_subset.columns],
              title = 'Periodic Contribution Growth on Initial Investment', 
              labels={"value": "Value", "variable": "Category", 'index': "Year"},
              )
  fig.update_layout(hovermode="x")
  return fig.show()

growth_df = periodic_contribution_growth(initial_investment, avg_annual_growth_rate, additional_contributions, contribution_rate, years)
display(growth_df)
bar_contr_growth(growth_df)


,Portfolio Worth,Total Return,Annual Contribution,Total Contribution,Initial Investment
1,1250.00,100.00,150.0,150.0,1000
2,1525.00,225.00,150.0,300.0,1000
3,1827.50,377.50,150.0,450.0,1000
4,2160.25,560.25,150.0,600.0,1000
5,2526.28,776.28,150.0,750.0,1000
6,2928.90,1028.90,150.0,900.0,1000
7,3371.79,1321.79,150.0,1050.0,1000
8,3858.97,1658.97,150.0,1200.0,1000
9,4394.87,2044.87,150.0,1350.0,1000
10,4984.36,2484.36,150.0,1500.0,1000
